## Decision Tree implementation using Entropy as the cost function

Mamello Maseko and 
Sandile Shongwe (1236067)

References:
https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [1]:
import numpy as np
import pandas as pd
from random import randrange
from random import seed
%xmode plain

Exception reporting mode: Plain


In [2]:
def gini_index(groups, classes):
    n_instances = float(np.sum([len(i) for i in groups]))
    gini_value = 0.0
    for grp in groups:
        size = float(len(grp))
        if size == 0:
            continue
        score = 0.0
        for val in classes:
            proportion = [ i[-1]  for i in grp]
            proportion = proportion.count(val)/size
            score += proportion * proportion
        gini_value += (1.0-score)*(size/n_instances)
    return gini_value

In [3]:
def informationGain(entropy_sys, groups, classes):
    tot_entropy = 0.0
    for grp in groups:
        if len(grp)==0:
            continue
        entropy = 0.0
        tmp_arr = (np.array(grp))
        prob = tmp_arr[:,-1].sum()/len(grp)
        entropy -= prob*np.log2(prob) +(1-prob)*np.log2((1-prob))
        tot_entropy += entropy
    tot_entropy = tot_entropy/(len(groups[0])+len(groups[1]))
    gain = entropy_sys - tot_entropy
    return gain

def get_split_g(data):

    idx_split = 100
    value_split = 100
    max_gain = -1000
    grp1 = None
    grp2 = None

    sys_prob = (np.array(data)[:,-1].sum())/len(data)
    sys_gain = -(sys_prob)*np.log2(sys_prob) - (1-sys_prob)*np.log2((1-sys_prob))
    for i in range(len(data[0])-1):
        for j in range(len(data)):
            tgrp1, tgrp2 = separate(i, data[j][i], data)
            tmp = informationGain(sys_prob,[tgrp1, tgrp2], [1,0])
            if tmp > max_gain:
                max_gain = tmp
                idx_split = i
                value_split = data[j][i]
                grp1 = tgrp1
                grp2 = tgrp2
    return {'idx':idx_split, 'val':value_split, 'gscore': max_gain, 'groups':[grp1, grp2] }


In [4]:
# this function splits the data into groups according to value 
#and the column index idx
def separate(idx, value, data):
    grp1 = []
    grp2 = []
    for instance in data:
        if instance[idx] < value:
            grp1.append(instance)
        else:
            grp2.append(instance)
    return grp1, grp2    

In [5]:
def end_of_split(group):
    positive = [row[-1] for row in group].count(1)
    negative = [row[-1] for row in group].count(0)
    if(positive > negative):
        return 1.0
    return 0.0

In [6]:
#correct
def split(node, max_depth, min_size, depth):
    groups = node['groups']
    left = groups[0]
    right = groups[1]
    del(node['groups'])
    
    if len(left) == 0 or len(right) == 0:
        node['right'] = end_of_split(left+right)
        node['left'] = end_of_split(left+right)
        return

    if depth >= max_depth:
        node['left'] = end_of_split(left)
        node['right'] = end_of_split(right)
        return 
    
    if len(left) <=  min_size:
        node['left'] = end_of_split(left)
    else:
        node['left'] = get_split_g(left)
        split(node['left'], max_depth, min_size, depth+1)
    
    if len(right) <=  min_size:
        node['right'] = end_of_split(right)
    else:
        node['right'] = get_split_g(right)
        split(node['right'], max_depth, min_size, depth+1)

def make_tree(train, max_depth, min_size):
    root = get_split_g(train)
    split(root, max_depth, min_size,1)
    return root

In [7]:
def predict(node, r):
    if r[node['idx']] < node['val']:
        if isinstance(node['left'], dict):
            return predict(node['left'], r)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], r)
        else:
            return node['right']

In [8]:
def decision_tree(train, test, max_depth, min_size):
    tr = build_tree(train, max_depth, min_size)
    lpredictions = list()
    for r in test:
        pr = predict(tr, r)
        lpredictions.append(pr)
    
    return lpredictions

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct/float(len(actual)) * 100.0

In [9]:
def l_csv(filename):
    data = pd.read_csv(filename, error_bad_lines=False)
    return data.values

def cross_validation_split(data, num_folds):
    data_split = list()
    data_cpy = list(data)
    fold_sz = int(len(data)/(num_folds))
    for i in range(num_folds):
        fold = list()
        while len(fold) < fold_sz:
            idx = randrange(len(data_cpy))
            fold.append(data_cpy.pop(idx))
        data_split.append(fold)
    return data_split       

def eval_algorithm(data, n_folds, max_depth, min_size):
    folds = cross_validation_split(data, n_folds)
    scores = list()
    for i in range(len(folds)):
        train_set = list(folds)
        train_set.pop(i)
        train_set = sum(train_set, [])
        test_set = list()
        for row in folds[i]:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = decision_tree(train_set, test_set, max_depth, min_size)
        actual = [row[-1] for row in folds[i]]
        acc = accuracy_metric(actual, predicted)
        scores.append(acc)
    return scores

#correct
def build_tree(train, max_depth, min_size):
    root = get_split_g(train)
    split(root, max_depth, min_size, 1)
    return root

In [10]:
seed(1)
df = pd.read_csv('data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] #drop unnamed column
df_xtrain = df.drop(['id','diagnosis'], axis=1)
x_train = df_xtrain.values
#  Note: Makes M = 1, B=0
df['diagnosis'] = np.unique(df['diagnosis'], return_index=True, return_inverse=True)[2]
y_train = df['diagnosis'].values
y_train.shape
dataset = np.hstack((x_train, y_train[:,np.newaxis]))
# dataset[:,-1]
n_folds = 5
max_depth = 7
min_size = 5
scores = eval_algorithm(dataset, n_folds, max_depth, min_size)

print('Scores: %s' % scores)

C:\Users\Sandile\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
C:\Users\Sandile\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


Scores: [93.80530973451327, 91.1504424778761, 90.2654867256637, 88.49557522123894, 92.92035398230088]


In [11]:
seed(1)
dataset1 = l_csv('data_banknote_authentication.csv')
dataset1.shape
dataset1[:,-1]
n_folds = 5
max_depth = 10
min_size = 3
scores = eval_algorithm(dataset1, n_folds, max_depth, min_size)
print('Scores: %s' % scores)

C:\Users\Sandile\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
C:\Users\Sandile\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


Scores: [77.00729927007299, 74.45255474452554, 79.56204379562044, 72.26277372262774, 74.81751824817519]
